In [12]:
#TODO LIST
#1- Change reading files to filter pdfs.
#2- Access to the image classification end point
#3- Create a JSON input for the endpoint (with the image)
#4- Retrieve prediction
#5- Store the prediction in a new BQ table


# Predictions Pipeline

## 1. Notebook Configuration
### 1.1. Loading Necessary Libraries

In [ ]:
# General libraries:
import os
#import glob
#import numpy as np
#import pandas as pd

# Dealing with images:
#import cv2
#import matplotlib.pyplot as plt

# Google APIs:
from google.cloud import storage



# Specific PDF libraries:
#!conda install -c conda-forge poppler
!sudo apt-get install -y poppler-data
!sudo apt-get install -y poppler-utils
!pip install pdf2image
from pdf2image import convert_from_path

### 1.2. Setting Notebook Inputs
#### 1.2.1 Google Cloud Settings

In [66]:
!gcloud config list

[core]
account = 136021895401-compute@developer.gserviceaccount.com
disable_usage_reporting = True
project = qwiklabs-gcp-00-373ac55d0e0a

Your active configuration is: [default]


In [21]:
PROJECT = 'qwiklabs-gcp-00-373ac55d0e0a'
REGION = 'us-central1'
BUCKET = 'qwiklabs-gcp-00-373ac55d0e0a'

DS_FOLDER = './dataset'
PDF_FOLDER = os.path.join(DS_FOLDER, 'pdf')
PNG_FOLDER = os.path.join(DS_FOLDER, 'png')
CSV_FOLDER = os.path.join(DS_FOLDER, 'csv')

os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION
os.environ['BUCKET'] = BUCKET

#### 1.2.2. Image Classification Endpoint

In [ ]:
IC_ENDPOINT_ID="7257673944809865216"
IC_PROJECT_ID="136021895401"
IC_INPUT_DATA_FILE="INPUT-JSON"

# Example of instance:
# {
#  "instances": [{
#    "content": "YOUR_IMAGE_BYTES"
#  }],
#   "parameters": {
#     "confidenceThreshold": 0.5,
#     "maxPredictions": 5
#   }
# }

#### 1.2.3. Object Detection Endpoint

In [ ]:
OD_ENDPOINT_ID="2074030773706424320"
OD_PROJECT_ID="136021895401"
OD_INPUT_DATA_FILE="INPUT-JSON"

# Example of instance:
# {
#  "instances": [{
#    "content": "YOUR_IMAGE_BYTES"
#  }],
#   "parameters": {
#     "confidenceThreshold": 0.5,
#     "maxPredictions": 5
#   }
# }

### 2. Accessing the PDFs of the Bucket

In [8]:
storage_client = storage.Client()

# Note: Client.list_blobs requires at least package version 1.17.0.
blobs = storage_client.list_blobs(BUCKET)

file_lst = []
for blob in blobs:
    if blob.name.find('labeled_patents/pdf/'):
        file_lst.append(blob.name)

In [29]:
# File name for testing hardcoded: TODO: ERASE
file_lst[0]
file_name = 'qwiklabs-gcp-00-373ac55d0e0a/labeled_patents/pdf/computer_vision_1.pdf'
file_name = r'gs://qwiklabs-gcp-00-373ac55d0e0a/labeled_patents/pdf/computer_vision_1.pdf'

In [32]:
# Donwoloading new pdf to a temporal file: TODO: How do we deal with this in a pipeline??
!gsutil -m cp gs://qwiklabs-gcp-00-373ac55d0e0a/labeled_patents/pdf/computer_vision_1.pdf ./temp
#!gsutil -m cp gs://gcs-public-data--labeled-patents/*.pdf  gs://labeled_patents/pdf

Copying gs://qwiklabs-gcp-00-373ac55d0e0a/labeled_patents/pdf/computer_vision_1.pdf...
/ [1/1 files][ 55.0 KiB/ 55.0 KiB] 100% Done                                    
Operation completed over 1 objects/55.0 KiB.                                     


In [51]:
# Transforming pdf into a png:
image = convert_from_path('./temp/computer_vision_1.pdf')
print(type(image))
image = image[0]
type(image)

<class 'list'>


PIL.PpmImagePlugin.PpmImageFile

In [54]:
# Encoding the image as a string of bytes:
import io

img_byte_arr = io.BytesIO()
image.save(img_byte_arr, format='PNG')
img_byte_arr = img_byte_arr.getvalue()


In [62]:
# Creating the instance for prediction in GCP:
instances = [
    {
      "instances": [{
                    "content": str(img_byte_arr)
                    }],
      "parameters": 
        {
        "confidenceThreshold": 0.5,
        "maxPredictions": 5
        }
}
]

In [ ]:

# EXAMPLE NOT TESTED - DONT RUN
input_path = 'qwiklabs-gcp-00-373ac55d0e0a/labeled_patents/pdf/computer_vision_1.pdf'
input_bucket_name = input_path.replace('gs://', '').split('/')[0]
input_bucket_name

  automl_client = automl.AutoMlClient.from_service_account_json(service_acct)
  model_full_id = automl_client.model_path(
      main_project_id,
      compute_region,
      model_id)
  prediction_client = automl.PredictionServiceClient.from_service_account_json(service_acct)

  # Create other clients
  storage_client = storage.Client.from_service_account_json(service_acct) 
  bq_client = bigquery.Client.from_service_account_json(service_acct)

In [64]:
from google.cloud import aiplatform
from google.protobuf import json_format

ENDPOINT_ID="2074030773706424320"

ENDPOINT_RESOURCENAME = ENDPOINT_ID
os.environ["ENDPOINT_RESOURCENAME"] = ENDPOINT_RESOURCENAME


# The AI Platform services require regional API endpoints.
api_endpoint = f'{REGION}-aiplatform.googleapis.com'
client_options = {"api_endpoint": api_endpoint}

# Initialize client that will be used to create and send requests.
# This client only needs to be created once, and can be reused for multiple requests.
client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)


instances = [json_format.ParseDict(instance, Value()) for instance in instances]
response = client.predict(endpoint=ENDPOINT_RESOURCENAME, instances=instances)

# The predictions are a google.protobuf.Value representation of the model's predictions.
print(" prediction:", response.predictions)



PermissionDenied: 403 Permission denied on resource project #2074030773706424320.